In [1]:
### importing all the libraries needed
import time
from time import sleep
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd

### Shopee Malaysia

In [2]:
%%time
### initial set up of selenium and chrome settings. Selenium is imitating as a Chrome broswer. 
### credits @ https://stackoverflow.com/questions/62057645/how-to-scrape-data-from-shopee-using-beautiful-soup

driver = webdriver.Chrome(ChromeDriverManager().install()) #this ensures you are using the latest chrome version everytime! Dont even need to find driver_path.  
chrome_options = Options()
chrome_options.headless = True
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")

# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

### get the browser load the url and display stuffs 
driver.get("https://shopee.com.my/shocking_sale")
driver.find_element_by_xpath('//button[text()="English"]').click()

### To get broswer to scroll endlessly. Credits @ https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### Data wrangling - selecting portions of interest from the css 
selenium_html = driver.page_source
soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 

sold_out = soup.find_all('div', class_ = "flash-sale-item-card flash-sale-item-card--landing-page flash-sale-item-card--MY flash-sale-item-card--sold-out")
still_selling = soup.find_all('div', class_ = "flash-sale-item-card flash-sale-item-card--landing-page flash-sale-item-card--MY")
items_type = [sold_out, still_selling]

# empty lists for loop below
items_name = []
original_price = []
sale_price = []
items_url = []
sold_quantity = []
web_list = []
sold_out_list = []

print("loop starting...")

# A double loop to get all fields of interest into lists. Credits @ QYGoh
for types in range(len(items_type)):
    for item in range(len(items_type[types])):
        items_name.append(items_type[types][item].find('div', class_ ="flash-sale-item-card__item-name-box").get_text())
        original_price.append(items_type[types][item].find('div', class_ = "flash-sale-item-card__original-price flash-sale-item-card__original-price--landing-page").get_text())
        sale_price.append(items_type[types][item].find('div', class_ = "flash-sale-item-card__current-price flash-sale-item-card__current-price--landing-page").get_text())
        
        # to get url- updated version on 05/08/2021
        links = items_type[types][item].find_all('a', {'class': "flash-sale-item-card-link", 'href': True})
        for link in links:
            items_url.append(link['href'])
        
        # special arrangement for sold quantity. Fully sold out have diff classes vs still selling
        if types == 0:
            sold_quantity.append(items_type[types][item].find('div', class_ = "flash-sale-sold-out flash-sale-sold-out--landing-page").get_text()) 
        elif types == 1:
            sold_quantity.append(items_type[types][item].find('div', class_ = "flash-sale-progress-bar__text").get_text())

print('printing df...')
            
df = pd.DataFrame(list(zip(items_name, original_price, sale_price, sold_quantity, items_url)),\
                 columns =['Name','Original_price','Sale_price','Sold_quantity','URL'])

print(df)
driver.quit() #to automatically close the browser

print('second loop starting...')
driver = webdriver.Chrome(ChromeDriverManager().install()) #this ensures you are using the latest chrome version everytime! Dont even need to find driver_path.  
chrome_options = Options()
chrome_options.headless = True
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")

# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

### get the browser load the url and display stuffs 
driver.get("https://shopee.com.my/shocking_sale")
driver.find_element_by_xpath('//button[text()="English"]').click()

### To get broswer to scroll endlessly. Credits @ https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# loop to obtain product categories. Credits @ shermwkm
links = [link.get_attribute('href') for link in driver.find_elements_by_class_name('flash-sale-item-card-link')]
for link in links:
    driver.get(link)
    sleep(30) # seconds. it is v impt to give browser some time to sleep, so that the scrolling can be completed and get full HTML
    selenium_html = driver.page_source
    soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
    if soup.find_all('div', class_ = '_3YfrdB'): # if product is sold out
        link1 = driver.find_element_by_class_name('link-to-keep-parent-style')
        link2 = link1.get_attribute('href')
        driver.get(link2)
        selenium_html = driver.page_source
        soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
        cat_so = [x.get_text() for x in soup.find_all(class_ = "_3QRNmL")]        
        sold_out_list.append(cat_so)
        my_sold_out_df = pd.DataFrame(sold_out_list)
        my_sold_out_df = my_sold_out_df.iloc[: , :4]
        my_sold_out_df.columns = ['Main Category','Sub Category','Product','Product Name']
    else: # if product is still in stock
        cat = [x.get_text() for x in soup.find_all('span', class_ = "_3QRNmL")]
        web_list.append(cat)
        my_in_stock_df = pd.DataFrame(web_list)
        my_in_stock_df = my_in_stock_df.iloc[: , :4]
        my_in_stock_df.columns = ['Main Category','Sub Category','Product','Product Name']

print('second loop ended...')
        
driver.quit() #to automatically close the browser

if len(sold_out_list) == 0:
    my_in_stock_df['Sold Out'] = 'No'
    my_df = my_in_stock_df 
else:
    my_sold_out_df['Sold Out'] = 'Yes'
    my_in_stock_df['Sold Out'] = 'No'
    my_df = my_sold_out_df.append(my_in_stock_df)
    
final_df = df.join(my_df)
final_df



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\sherm\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


loop starting...
printing df...
                                                 Name Original_price  \
0   Skechers Men Sport Casual Courtwald Sandals - ...       RM 79.00   
1   Skechers Women Side Lines 2 Cali Sandals - 873...       RM 99.00   
2   Skechers Women Side Lines 2 Cali Sandals - 873...       RM 99.00   
3   OPPO A15s Smartphone | 4GB RAM + 64GB ROM | AI...      RM 599.00   
4   NIVEA Female Deodorant Spray - Hokkaido Rose (...       RM 28.60   
5   KONKA 720 Adjustment Portable Hand Mini Clip C...       RM 59.00   
6   APACHE TurboSpray HS-04E Portable Rechargeable...      RM 139.00   
7   Schneider Electric Neo 10A 1 Gang 1 way Switch...       RM 59.40   
8   Schneider Electric Vivace 16AX 1 Gang 2-Way Sw...        RM 9.30   
9   Schneider Electric Vivace 10A 1 Gang Bell Pres...        RM 9.20   
10     CLiPtec Single USB Port 1A Home Charger GZU360       RM 35.00   
11  CLiPtec Multimedia Dual Drivers Table Stand Mi...       RM 49.00   
12  CLiPtec U-SOUNDMATE USB2.0 N



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\sherm\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


second loop starting...
second loop ended...
Wall time: 19min 7s


,Name,Original_price,Sale_price,Sold_quantity,URL,Main Category,Sub Category,Product,Product Name,Sold Out
0,Skechers Men Sport Casual Courtwald Sandals - ...,RM 79.00,RM 75.10,4 sold,/Skechers-Men-Sport-Casual-Courtwald-Sandals-8...,Men Shoes,Sandals & Flip Flops,Sandals,Skechers Men Sport Casual Courtwald Sandals - ...,No
1,Skechers Women Side Lines 2 Cali Sandals - 873...,RM 99.00,RM 65.60,5 sold,/Skechers-Women-Side-Lines-2-Cali-Sandals-8730...,Women Shoes,Flat Sandals & Flip Flops,Flat Sandals,Skechers Women Side Lines 2 Cali Sandals - 873...,No
2,Skechers Women Side Lines 2 Cali Sandals - 873...,RM 99.00,RM 65.60,5 sold,/Skechers-Women-Side-Lines-2-Cali-Sandals-8730...,Women Shoes,Flat Sandals & Flip Flops,Flat Sandals,Skechers Women Side Lines 2 Cali Sandals - 873...,No
3,OPPO A15s Smartphone | 4GB RAM + 64GB ROM | AI...,RM 599.00,RM 499.00,0 sold,/OPPO-A15s-Smartphone-4GB-RAM-64GB-ROM-AI-Trip...,Mobile & Gadgets,Mobile Phones,OPPO A15s Smartphone | 4GB RAM + 64GB ROM | AI...,Mobile & Gadgets,No
4,NIVEA Female Deodorant Spray - Hokkaido Rose (...,RM 28.60,RM 22.32,7 sold,/NIVEA-Female-Deodorant-Spray-Hokkaido-Rose-(2...,Beauty,Bath & Body Care,Body Deodorants,NIVEA Female Deodorant Spray - Hokkaido Rose (...,No
5,KONKA 720 Adjustment Portable Hand Mini Clip C...,RM 59.00,RM 22.90,9 sold,/KONKA-720-Adjustment-Portable-Hand-Mini-Clip-...,Home Appliances,Large Household Appliances,Cooling,KONKA 720 Adjustment Portable Hand Mini Clip C...,No
6,APACHE TurboSpray HS-04E Portable Rechargeable...,RM 139.00,RM 125.00,1 sold,/APACHE-TurboSpray-HS-04E-Portable-Rechargeabl...,Home & Living,Tools & Home Improvement,"Water Pumps, Parts & Accessories",APACHE TurboSpray HS-04E Portable Rechargeable...,No
7,Schneider Electric Neo 10A 1 Gang 1 way Switch...,RM 59.40,RM 53.00,0 sold,/Schneider-Electric-Neo-10A-1-Gang-1-way-Switc...,Home & Living,Tools & Home Improvement,Tools,Schneider Electric Neo 10A 1 Gang 1 way Switch...,No
8,Schneider Electric Vivace 16AX 1 Gang 2-Way Sw...,RM 9.30,RM 8.80,2 sold,/Schneider-Electric-Vivace-16AX-1-Gang-2-Way-S...,Home & Living,Tools & Home Improvement,Tools,Schneider Electric Vivace 16AX 1 Gang 2-Way Sw...,No
9,Schneider Electric Vivace 10A 1 Gang Bell Pres...,RM 9.20,RM 8.80,1 sold,/Schneider-Electric-Vivace-10A-1-Gang-Bell-Pre...,Home & Living,Tools & Home Improvement,Tools,Schneider Electric Vivace 10A 1 Gang Bell Pres...,No


In [4]:
# saving excel file named according to datetime of extraction 
timestr = time.strftime("%Y%m%d-%H%M")
final_df.to_excel('Shopee_MY'+ timestr + '.xlsx')

### Shopee Singapore

In [7]:
%%time
### initial set up of selenium and chrome settings. Selenium is imitating as a Chrome broswer. 
### credits @ https://stackoverflow.com/questions/62057645/how-to-scrape-data-from-shopee-using-beautiful-soup

driver = webdriver.Chrome(ChromeDriverManager().install()) #this ensures you are using the latest chrome version everytime! Dont even need to find driver_path.  
chrome_options = Options()
chrome_options.headless = True
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")

# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

### get the browser load the url and display stuffs 
driver.get("https://shopee.sg/flash_deals")

### To get broswer to scroll endlessly. Credits @ https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### Data wrangling - selecting portions of interest from the css 
selenium_html = driver.page_source
soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 

sold_out = soup.find_all('div', class_ = "flash-sale-item-card flash-sale-item-card--landing-page flash-sale-item-card--SG flash-sale-item-card--sold-out")
still_selling = soup.find_all('div', class_ = "flash-sale-item-card flash-sale-item-card--landing-page flash-sale-item-card--SG")
items_type = [sold_out, still_selling]

# empty lists for loop below
items_name = []
original_price = []
sale_price = []
items_url = []
sold_quantity = []
web_list = []
sold_out_list = []

print("loop starting...")

# A double loop to get all fields of interest into lists. Credits @ QYGoh
for types in range(len(items_type)):
    for item in range(len(items_type[types])):
        items_name.append(items_type[types][item].find('div', class_ ="flash-sale-item-card__item-name-box").get_text())
        original_price.append(items_type[types][item].find('div', class_ = "flash-sale-item-card__original-price flash-sale-item-card__original-price--landing-page").get_text())
        sale_price.append(items_type[types][item].find('div', class_ = "flash-sale-item-card__current-price flash-sale-item-card__current-price--landing-page").get_text())
        
        # to get url- updated version on 05/08/2021
        links = items_type[types][item].find_all('a', {'class': "flash-sale-item-card-link", 'href': True})
        for link in links:
            items_url.append(link['href'])
        
        # special arrangement for sold quantity. Fully sold out have diff classes vs still selling
        if types == 0:
            sold_quantity.append(items_type[types][item].find('div', class_ = "flash-sale-sold-out flash-sale-sold-out--landing-page").get_text()) 
        elif types == 1:
            sold_quantity.append(items_type[types][item].find('div', class_ = "flash-sale-progress-bar__text").get_text())

print('printing df...')
            
df = pd.DataFrame(list(zip(items_name, original_price, sale_price, sold_quantity, items_url)),\
                 columns =['Name','Original_price','Sale_price','Sold_quantity','URL'])

print(df)
driver.quit() #to automatically close the browser

print('second loop starting...')
driver = webdriver.Chrome(ChromeDriverManager().install()) #this ensures you are using the latest chrome version everytime! Dont even need to find driver_path.  
chrome_options = Options()
chrome_options.headless = True
chrome_options.add_argument("--window-size=1920,1200")
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\username\\AppData\\Local\\Google\\Chrome\\User Data\\Default')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")

# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

### get the browser load the url and display stuffs 
driver.get("https://shopee.sg/flash_deals")

### To get broswer to scroll endlessly. Credits @ https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# loop to obtain product categories. Credits @ shermwkm
links = [link.get_attribute('href') for link in driver.find_elements_by_class_name('flash-sale-item-card-link')]
for link in links:
    driver.get(link)
    sleep(15) # seconds. it is v impt to give browser some time to sleep, so that the scrolling can be completed and get full HTML
    selenium_html = driver.page_source
    soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
    if soup.find_all('div', class_ = '_3YfrdB'): # if product is sold out
        link1 = driver.find_element_by_class_name('link-to-keep-parent-style')
        link2 = link1.get_attribute('href')
        driver.get(link2)
        selenium_html = driver.page_source
        soup = BeautifulSoup(selenium_html, 'lxml') #lxml parsing is faster than html parsing 
        cat_so = [x.get_text() for x in soup.find_all(class_ = "_3QRNmL")]        
        sold_out_list.append(cat_so)
        sg_sold_out_df = pd.DataFrame(sold_out_list)
        sg_sold_out_df = my_sold_out_df.iloc[: , :4]
        sg_sold_out_df.columns = ['Main Category','Sub Category','Product','Product Name']
    else: # if product is still in stock
        cat = [x.get_text() for x in soup.find_all('span', class_ = "_3QRNmL")]
        web_list.append(cat)
        sg_in_stock_df = pd.DataFrame(web_list)
        sg_in_stock_df = my_in_stock_df.iloc[: , :4]
        sg_in_stock_df.columns = ['Main Category','Sub Category','Product','Product Name']

print('second loop ended...')
        
driver.quit() #to automatically close the browser

if len(sold_out_list) == 0:
    sg_in_stock_df['Sold Out'] = 'No'
    sg_df = sg_in_stock_df 
else:
    sg_sold_out_df['Sold Out'] = 'Yes'
    sg_in_stock_df['Sold Out'] = 'No'
    sg_df = sg_sold_out_df.append(sg_in_stock_df)
    
sg_final_df = df.join(sg_df)
sg_final_df



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\sherm\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


loop starting...
printing df...
                                                 Name Original_price  \
0   Bright  Home Rubber Insulation Pads Anti-scald...         $ 8.99   
1   Waterproof Laptop Keyboard protective 13 14 15...         $ 5.50   
2                  EAT- Handmade Ngoh Hiang (1 piece)         $ 1.80   
3   HAEVR Toothpaste squeezer, household bathroom ...         $ 9.98   
4   Febreze With Ambi Pur Fabric Refresher Spray 3...         $ 4.90   
5   Bright  Home 4-piece set Acne Needle Blackhead...         $ 9.99   
6   MUMENG Portable Sharpener Sharpener Mini Creat...         $ 8.99   
7   6Pcs/boxed INS Press Gel Pen Student Cute Pres...        $ 15.00   
8   Samsung Galaxy Watch Active2 BT Aluminium 40mm...       $ 318.00   
9   HUAWEI MateBook D15 Laptop / Intel i3  / 8GB R...       $ 798.00   
10  9Pcs/lot colorful Ink Color Photo Album Gel Pe...         $ 3.57   
11  HP All-in-One Desktop PC 27-dp1120d / 11th Gen...     $ 1,899.00   
12   adidas ORIGINALS Track Jack



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\sherm\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


second loop starting...
second loop ended...
Wall time: 9min 44s


C:\Users\sherm\anaconda3\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Name,Original_price,Sale_price,Sold_quantity,URL,Main Category,Sub Category,Product,Product Name,Sold Out
0,Bright Home Rubber Insulation Pads Anti-scald...,$ 8.99,$ 0.99,selling out,/Bright-Home-Rubber-Insulation-Pads-Anti-scald...,Home & Living,Dinnerware,Placemats & Coasters,Bright Home Rubber Insulation Pads Anti-scald...,No
1,Waterproof Laptop Keyboard protective 13 14 15...,$ 5.50,$ 0.99,58 sold,/Waterproof-Laptop-Keyboard-protective-13-14-1...,Computers & Accessories,Peripherals & Accessories,Keyboard & Trackpad Covers,Waterproof Laptop Keyboard protective 13 14 15...,No
2,EAT- Handmade Ngoh Hiang (1 piece),$ 1.80,$ 0.90,567 sold,https://shopee.sg/dp,None,None,None,None,No
3,"HAEVR Toothpaste squeezer, household bathroom ...",$ 9.98,$ 0.99,92 sold,/HAEVR-Toothpaste-squeezer-household-bathroom-...,Home & Living,Bathrooms,Toothbrush Holders & Toothpaste Dispensers,"HAEVR Toothpaste squeezer, household bathroom ...",No
4,Febreze With Ambi Pur Fabric Refresher Spray 3...,$ 4.90,$ 3.90,35 sold,/Febreze-With-Ambi-Pur-Fabric-Refresher-Spray-...,Home & Living,Home Fragrance & Aromatherapy,Air Fresheners & Home Fragrance,Febreze With Ambi Pur Fabric Refresher Spray 3...,No
5,Bright Home 4-piece set Acne Needle Blackhead...,$ 9.99,$ 0.99,80 sold,/Bright-Home-4-piece-set-Acne-Needle-Blackhead...,Beauty,Skincare,Acne Treatment,Bright Home 4-piece set Acne Needle Blackhead...,No
6,MUMENG Portable Sharpener Sharpener Mini Creat...,$ 8.99,$ 0.99,121 sold,/MUMENG-Portable-Sharpener-Sharpener-Mini-Crea...,Stationery,School & Office Equipment,Sharpeners,MUMENG Portable Sharpener Sharpener Mini Creat...,No
7,6Pcs/boxed INS Press Gel Pen Student Cute Pres...,$ 15.00,$ 1.88,121 sold,/6pcs-boxed-INS-Press-Gel-Pen-Student-Cute-Pre...,Stationery,Writing & Correction,Pens & Inks,6pcs/boxed INS Press Gel Pen Student Cute Pres...,No
8,Samsung Galaxy Watch Active2 BT Aluminium 40mm...,$ 318.00,$ 218.00,6 sold,/Samsung-Galaxy-Watch-Active2-BT-Aluminium-40m...,Mobile & Gadgets,Wearable Devices,Smartwatches & Fitness Trackers,Samsung Galaxy Watch Active2 BT Aluminium 40mm...,No
9,HUAWEI MateBook D15 Laptop / Intel i3 / 8GB R...,$ 798.00,$ 698.00,13 sold,/HUAWEI-MateBook-D15-Laptop-Intel-i3-8GB-Ram-2...,Computers & Accessories,Laptops,HUAWEI MateBook D15 Laptop / Intel i3 / 8GB R...,Computers & Accessories,No


In [8]:
# saving excel file named according to datetime of extraction 
timestr = time.strftime("%Y%m%d-%H%M")
sg_final_df.to_excel('Shopee_SG'+ timestr + '.xlsx')